In [ ]:
import arcpy

In [ ]:
#converting mangrove boundaries into points (generate points along line)
arcpy.management.GeneratePointsAlongLines(
    Input_Features="gmw_v3_2020_vec",
    Output_Feature_Class=r"C:\Users\Administrator\Documents\ArcGIS\Projects\mangrove_width\mangrove_distance_v2.gdb\gmw_v3_2020_vec_GeneratePointsAlongLines",
    Point_Placement="DISTANCE",
    Distance="20 Meters",
    Percentage=None,
    Include_End_Points="NO_END_POINTS",
    Add_Chainage_Fields="NO_CHAINAGE"
)

In [ ]:
#importing zillow data into arcgis pro
arcpy.management.XYTableToPoint(
    in_table="all_mangrove_data_propertyfe.csv",
    out_feature_class=r"C:\Users\Administrator\Documents\ArcGIS\Projects\mangrove_width\mangrove_distance_v2.gdb\propertyfe_id",
    x_field="longitude",
    y_field="latitude",
    z_field=None,
    coordinate_system='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision'
)

In [ ]:
#calculate x,y of zillow housing data in a regional projection so I can do planar calculations later
arcpy.management.CalculateGeometryAttributes(
    in_features="propertyfe_id",
    geometry_property="x_house POINT_X;y_house POINT_Y",
    length_unit="",
    area_unit="",
    coordinate_system='PROJCS["NAD_1983_2011_Florida_GDL_Albers",GEOGCS["GCS_NAD_1983_2011",DATUM["D_NAD_1983_2011",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["False_Easting",400000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-84.0],PARAMETER["Standard_Parallel_1",24.0],PARAMETER["Standard_Parallel_2",31.5],PARAMETER["Latitude_Of_Origin",24.0],UNIT["Meter",1.0]]',
    coordinate_format="SAME_AS_INPUT"
)

In [ ]:
#finding closest mangrove point to each house
arcpy.analysis.SpatialJoin(
    target_features="propertyfe_id",
    join_features="gmw_v3_2020_vec_GeneratePointsAlongLines",
    out_feature_class=r"C:\Users\Administrator\Documents\ArcGIS\Projects\mangrove_width\mangrove_distance_v2.gdb\propertyfe_id_mangrove_points",
    join_operation="JOIN_ONE_TO_ONE",
    join_type="KEEP_COMMON",
    field_mapping='Field1 "Field1" true true false 4 Long 0 0,First,#,propertyfe_id,Field1,-1,-1;transid "transid" true true false 4 Long 0 0,First,#,propertyfe_id,transid,-1,-1;latitude "latitude" true true false 8 Double 0 0,First,#,propertyfe_id,latitude,-1,-1;longitude "longitude" true true false 8 Double 0 0,First,#,propertyfe_id,longitude,-1,-1;house_id "house_id" true true false 8000 Text 0 0,First,#,propertyfe_id,house_id,0,8000;house_id_X "house_id_X" true true false 8 Double 0 0,First,#,propertyfe_id,house_id_X,-1,-1;house_id_Y "house_id_Y" true true false 8 Double 0 0,First,#,propertyfe_id,house_id_Y,-1,-1;x_house "x_house" true true false 8 Double 0 0,First,#,propertyfe_id,x_house,-1,-1;y_house "y_house" true true false 8 Double 0 0,First,#,propertyfe_id,y_house,-1,-1;ORIG_FID "ORIG_FID" true true false 4 Long 0 0,First,#,gmw_v3_2020_vec_GeneratePointsAlongLines,ORIG_FID,-1,-1;PXLVAL "PXLVAL" true true false 4 Long 0 0,First,#,gmw_v3_2020_vec_GeneratePointsAlongLines,PXLVAL,-1,-1;Shape_Length "Shape_Length" false true false 8 Double 0 0,First,#,gmw_v3_2020_vec_GeneratePointsAlongLines,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true false 8 Double 0 0,First,#,gmw_v3_2020_vec_GeneratePointsAlongLines,Shape_Area,-1,-1;x_mangrove "x_mangrove" true true false 8 Double 0 0,First,#,gmw_v3_2020_vec_GeneratePointsAlongLines,x_mangrove,-1,-1;y_mangrove "y_mangrove" true true false 8 Double 0 0,First,#,gmw_v3_2020_vec_GeneratePointsAlongLines,y_mangrove,-1,-1',
    match_option="CLOSEST_GEODESIC",
    search_radius=None,
    distance_field_name="distance_geo"
)

In [ ]:
#getting angle between house and mangrove point
arcpy.management.CalculateField(
    in_table="propertyfe_id_mangrove_points",
    field="theta",
    expression="getAngle(!x_house!,!y_house!,!x_mangrove!,!y_mangrove!)",
    expression_type="PYTHON3",
    code_block="""import math
def getAngle(x1,y1,x2,y2):
    angle =  math.atan2(x2-x1,y2-y1) * 180/math.pi
    if angle < 0:
        return( 360+angle )
    else :
        return( angle )""",
    field_type="TEXT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

In [ ]:
#get a x-coordinate that is past the closest mangrove distance exactly 2,000 meters (it is the exact same angle as between house and mangrove point)
arcpy.management.CalculateField(
    in_table="propertyfe_id_mangrove_points",
    field="x_transect_terminus",
    expression="get_x_coordinate(!distance_geo!+2000, !theta!, 0, !x_mangrove!)",
    expression_type="PYTHON3",
    code_block="""import math
def get_x_coordinate (l, gamma, theta, x):
    ''' convert gamma and theta to radians '''
    arg = gamma + theta
    return l * math.sin(arg*math.pi/180) + x""",
    field_type="TEXT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

In [ ]:
#get a y-coordinate that is past the closest mangrove distance exactly 2,000 meters (it is the exact same angle as between house and mangrove point)
arcpy.management.CalculateField(
    in_table="propertyfe_id_mangrove_points",
    field="y_transect_terminus",
    expression="get_y_coordinate(!distance_geo!+2000, !theta!, 0, !y_mangrove!)",
    expression_type="PYTHON3",
    code_block="""import math
def get_y_coordinate (l, gamma, theta, y):
    ''' convert gamma and theta to radians '''
    arg = gamma + theta
    return l * math.cos(arg*math.pi/180) + y""",
    field_type="TEXT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

In [ ]:
#create a line between the house and the x,y point through and past the closest mangrove point
arcpy.management.XYToLine(
    in_table="propertyfe_id_mangrove_points",
    out_featureclass=r"C:\Users\Administrator\Documents\ArcGIS\Projects\mangrove_width\mangrove_distance_v2.gdb\mangrove_transect_raw",
    startx_field="x_house",
    starty_field="y_house",
    endx_field="x_transect_terminus",
    endy_field="y_transect_terminus",
    line_type="PLANAR",
    id_field="house_id",
    spatial_reference='PROJCS["NAD_1983_2011_Florida_GDL_Albers",GEOGCS["GCS_NAD_1983_2011",DATUM["D_NAD_1983_2011",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["False_Easting",400000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-84.0],PARAMETER["Standard_Parallel_1",24.0],PARAMETER["Standard_Parallel_2",31.5],PARAMETER["Latitude_Of_Origin",24.0],UNIT["Meter",1.0]];-19550100 -7526400 10000;-100000 10000;-100000 10000;0.001;0.001;0.001;IsHighPrecision',
    attributes="NO_ATTRIBUTES"
)

In [ ]:
#clip the prior transect line so that we only get the line portions that intersect the global mangrove watch polygons
with arcpy.EnvManager(parallelProcessingFactor="50%"):
    arcpy.analysis.PairwiseClip(
        in_features="mangrove_transect_raw",
        clip_features="gmw_v3_2020_florida",
        out_feature_class=r"C:\Users\Administrator\Documents\ArcGIS\Projects\mangrove_width\mangrove_distance_v2.gdb\mangrove_transect_intersect",
        cluster_tolerance=None
    )

In [ ]:
#get the number of pieces of each transect
arcpy.management.CalculateGeometryAttributes(
    in_features="mangrove_transect_intersect",
    geometry_property="parts PART_COUNT",
    length_unit="",
    area_unit="",
    coordinate_system=None,
    coordinate_format="SAME_AS_INPUT"
)

In [ ]:
#we only want those transect lines that have more than one part, because this means it fully intersected a mangrove and a gap, so that it's entire length is being measured
arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="mangrove_transect_intersect",
    selection_type="NEW_SELECTION",
    where_clause="parts > 1",
    invert_where_clause=None
)

In [ ]:
#explode attributes
#there is not arcpy code for this portion, you just select all the multipart features and in the edit ribbon you click explode, let it run for a while, and then save afterwards

In [ ]:
#saving a copy of the exploded features for safe keeping
arcpy.management.CopyFeatures(
    in_features="mangrove_transect_intersect",
    out_feature_class=r"C:\Users\Administrator\Documents\ArcGIS\Projects\mangrove_width\mangrove_distance_v2.gdb\mangrove_transect_intersect_exploded",
    config_keyword="",
    spatial_grid_1=None,
    spatial_grid_2=None,
    spatial_grid_3=None
)

In [ ]:
#getting planar length of all the transect parts
arcpy.management.CalculateGeometryAttributes(
    in_features="mangrove_transect_intersect_exploded",
    geometry_property="x_start LINE_START_X;y_start LINE_START_Y;x_end LINE_END_X;y_end LINE_END_Y",
    length_unit="METERS",
    area_unit="",
    coordinate_system='PROJCS["NAD_1983_2011_Florida_GDL_Albers",GEOGCS["GCS_NAD_1983_2011",DATUM["D_NAD_1983_2011",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["False_Easting",400000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-84.0],PARAMETER["Standard_Parallel_1",24.0],PARAMETER["Standard_Parallel_2",31.5],PARAMETER["Latitude_Of_Origin",24.0],UNIT["Meter",1.0]]',
    coordinate_format="SAME_AS_INPUT"
)

In [ ]:
#getting the geodesic distance for all the transect parts (in case your analysis calls for this)
arcpy.management.CalculateGeometryAttributes(
    in_features="mangrove_transect_intersect_exploded",
    geometry_property="mangrove_width_meters LENGTH_GEODESIC",
    length_unit="METERS",
    area_unit="",
    coordinate_system='PROJCS["NAD_1983_2011_Florida_GDL_Albers",GEOGCS["GCS_NAD_1983_2011",DATUM["D_NAD_1983_2011",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["False_Easting",400000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-84.0],PARAMETER["Standard_Parallel_1",24.0],PARAMETER["Standard_Parallel_2",31.5],PARAMETER["Latitude_Of_Origin",24.0],UNIT["Meter",1.0]]',
    coordinate_format="SAME_AS_INPUT"
)

In [ ]:
#exporting the data as a table for the final step in R
arcpy.conversion.ExportTable(
    in_table="mangrove_transect_intersect_exploded",
    out_table=r"C:\Users\Administrator\Documents\ArcGIS\Projects\mangrove_width\mangrove_width_v2\mangrove_transect_intersect_all.csv",
    where_clause="",
    use_field_alias_as_name="NOT_USE_ALIAS",
    field_mapping='x_house "x_house" true true false 8 Double 0 0,First,#,mangrove_transect_intersect_exploded,x_house,-1,-1;y_house "y_house" true true false 8 Double 0 0,First,#,mangrove_transect_intersect_exploded,y_house,-1,-1;x_transect_terminus "x_transect_terminus" true true false 8 Double 0 0,First,#,mangrove_transect_intersect_exploded,x_transect_terminus,-1,-1;y_transect_terminus "y_transect_terminus" true true false 8 Double 0 0,First,#,mangrove_transect_intersect_exploded,y_transect_terminus,-1,-1;house_id "house_id" true true false 8000 Text 0 0,First,#,mangrove_transect_intersect_exploded,house_id,0,8000;parts "parts" true true false 4 Long 0 0,First,#,mangrove_transect_intersect_exploded,parts,-1,-1;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,mangrove_transect_intersect_exploded,Shape_Length,-1,-1;x_start "x_start" true true false 8 Double 0 0,First,#,mangrove_transect_intersect_exploded,x_start,-1,-1;y_start "y_start" true true false 8 Double 0 0,First,#,mangrove_transect_intersect_exploded,y_start,-1,-1;x_end "x_end" true true false 8 Double 0 0,First,#,mangrove_transect_intersect_exploded,x_end,-1,-1;y_end "y_end" true true false 8 Double 0 0,First,#,mangrove_transect_intersect_exploded,y_end,-1,-1',
    sort_field=None
)

In [ ]:
#the next part of the code is done in R where you get the distance between each of the line's start and end points and each house it's associated with and then you only keep the one that is closest.
#right now we have many pieces for each home and we only need the closest one